# **Preparation**

``note: Make sure your laptop/PC support TeX``

## **Import Library**

In [ ]:
# Install
!pip install PyDOE
!pip install tensorflow==1.15

In [ ]:
# Import
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import random
import scipy.io
import tensorflow as tf
import time
import timeit
import math as m
import scipy.interpolate

from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyDOE import lhs


In [ ]:
#! sudo apt-get install texlive-latex-recommended 
#! sudo apt install texlive-latex-extra
#! sudo apt install dvipng
#! sudo apt install cm-super


## **Class & Functions**

### **PINN Class**

In [ ]:
# @tf.function
class Pinn:

    def __init__(self, data, params, exist_model=False, file_dir=""):
        self.unpack(data, params)
        self.initialize_variables()

        # Initialize Neural Network Computational Graph
        # Weight & biases
        if exist_model:
            print("Loading NN parameters ...")
            self.weights, self.biases = self.load_model(file_dir)
        else:
            self.weights, self.biases = self.initialize_network()
        
        # Placeholder & Graph
        # Placeholder (where we put input)
        self.initialize_placeholders()

        # Computational graph of Physics-Informed
        self.graph_network()

        # Computational graph of loss
        self.graph_loss()
        
        # Optimizers
        self.initialize_optimizers()

        # Session
        self.initialize_session()

    def callback(self, loss_test, loss_total, loss_collo, loss_neu, loss_dir):
        self.count += 1
        self.loss_test_log.append(loss_test)
        self.loss_total_log.append(loss_total)
        self.loss_collo_log.append(loss_collo)
        self.loss_dir_log.append(loss_dir)
        self.loss_neu_log.append(loss_neu)
        
        if self.count % self.verboses_newton == 0:    
            print("iter: %d, Loss_Test: %.4e, Loss Total: %.4e, Loss Collo: %.4e, Loss Neumann: %.4e, Loss Dirichlet: %.4e" %
                  (self.count, loss_test, loss_total, loss_collo, loss_neu, loss_dir))

    def normalize_input_data(self):
        # Normalize data
        x_c = self.normalize_data(data=self.x_c, axis="x")
        y_c = self.normalize_data(data=self.y_c, axis="y")
        x_left = self.normalize_data(data=self.x_left, axis="x")
        y_left = self.normalize_data(data=self.y_left, axis="y")
        x_right = self.normalize_data(data=self.x_right, axis="x")
        y_right = self.normalize_data(data=self.y_right, axis="y")
        x_lower = self.normalize_data(data=self.x_lower, axis="x")
        y_lower = self.normalize_data(data=self.y_lower, axis="y")
        x_upper = self.normalize_data(data=self.x_upper, axis="x")
        y_upper = self.normalize_data(data=self.y_upper, axis="y")
        x_test = self.normalize_data(data=self.x_test, axis="x")
        y_test = self.normalize_data(data=self.y_test, axis="y")

        return x_c, y_c, x_left, y_left, x_right, y_right, x_lower, y_lower, x_upper, y_upper, x_test, y_test

    def fit_newton(self):
        # Change flag
        self.newton_started = True

        # Normalize data
        (x_c, y_c, 
         x_left, y_left, 
         x_right, y_right, 
         x_lower, y_lower,
         x_upper, y_upper,
         x_test, y_test) = self.normalize_input_data()

        tf_dict = {self.x_c_tf: x_c, self.y_c_tf: y_c,                          # collocation data
                   self.x_left_tf: x_left, self.y_left_tf: y_left,              # left data (pts)
                   self.u_left_tf: self.u_left, 
                   self.x_right_tf: x_right, self.y_right_tf: y_right,          # right data
                   self.u_right_tf: self.u_right,
                   self.x_lower_tf: x_lower, self.y_lower_tf: y_lower,          # lower data
                   self.u_lower_tf: self.u_lower,  
                   self.x_upper_tf: x_upper, self.y_upper_tf: y_upper,          # upper data
                   self.u_upper_tf: self.u_upper,
                   self.x_test_tf: x_test, self.y_test_tf: y_test,              # test data
                   self.u_test_tf: self.u_test}
               
        self.train_op_newton.minimize(self.sess,
                                      feed_dict=tf_dict,
                                      fetches=[self.loss_test, self.loss_total,
                                              self.loss_collo, self.loss_neu, 
                                              self.loss_dir],
                                      loss_callback=self.callback)

    def grad_finder(self):
        self.grad_x_left = tf.gradients(self.u_left_pred, self.x_left_tf)[0] / self.sigma_x
        self.grad_x_right = tf.gradients(self.u_right_pred, self.x_right_tf)[0] / self.sigma_x
        self.grad_y_lower = tf.gradients(self.u_lower_pred, self.y_lower_tf)[0] / self.sigma_y

    def graph_loss(self):
        # Test
        self.loss_test = tf.math.sqrt(tf.reduce_mean(tf.square(self.u_test_tf - self.u_test_pred)))

        # Collocation points
        self.loss_collo = tf.reduce_mean(tf.square(self.f_pred_u)) 
        
        # Boundary
        self.grad_finder()
        self.loss_dir = tf.reduce_mean(tf.square(self.u_upper_pred-self.u_upper_tf))
        self.loss_neu = tf.reduce_mean(tf.square(self.grad_x_left-self.u_left_tf)) \
                        + tf.reduce_mean(tf.square(self.grad_x_right-self.u_right_tf)) \
                        + tf.reduce_mean(tf.square(self.grad_y_lower-self.u_lower_tf))

        self.loss_collo = self.coef_bc_val*(self.loss_collo)

        self.loss_total = self.loss_neu + self.loss_collo + self.loss_dir

    def graph_network(self):
        # Test data
        (self.u_test_pred) = self.net_dnn(self.x_test_tf, self.y_test_tf)
        
        # Predict data
        (self.u_pred) = self.net_dnn(self.x_tf, self.y_tf)
        
        # Collocation points
        (self.f_pred_u) = self.net_physics(self.x_c_tf, self.y_c_tf)
        
        # left
        (self.u_left_pred) = self.net_dnn(self.x_left_tf, self.y_left_tf)
        
        # right
        (self.u_right_pred) = self.net_dnn(self.x_right_tf, self.y_right_tf)
        
        # Lower
        (self.u_lower_pred) = self.net_dnn(self.x_lower_tf, self.y_lower_tf)
        
        # Upper
        (self.u_upper_pred) = self.net_dnn(self.x_upper_tf,self.y_upper_tf)

    def load_model(self, file_dir):
        weights = []
        biases = []
        num_layers = len(self.layers)
        
        with open(file_dir, 'rb') as f:
            dnn_weights, dnn_biases = pickle.load(f)

            # stored model mush has the same layers
            assert num_layers == (len(dnn_weights)+1)

            for num in range(0, num_layers-1):
                W = tf.Variable(dnn_weights[num])
                b = tf.Variable(dnn_biases[num])
                weights.append(W)
                biases.append(b)
                print('Loaded NN parameters successfully ...')
            

        return weights, biases

    def initialize_network(self):
        # Initialize
        weights = []
        biases = []
        num_layers = len(self.layers)
        layers = self.layers

        # Create network
        for lyr in range(num_layers-1):
            # initialize weights from Xavier initialization
            np.random.seed(self.random_seed)
            W = self.xavier_init(size=[layers[lyr], 
                                       layers[lyr+1]])

            # initialize biases = 0
            np.random.seed(self.random_seed)
            b = tf.Variable(tf.zeros([1, layers[lyr+1]],
                                     dtype=tf.float32),
                            dtype=tf.float32)

            # Append generated weights & biases to the list
            weights.append(W)
            biases.append(b)

        return weights, biases

    def initialize_optimizers(self):
        self.train_op_newton = tf.contrib.opt.ScipyOptimizerInterface(
                                    self.loss_total,
                                    var_list = self.weights + self.biases,
                                    method = "L-BFGS-B",
                                    options = {"maxiter": 100000,
                                               "maxfun": 10000,
                                               "maxcor": 50,
                                               "maxls": 50,
                                               "ftol": 1e8*np.finfo(float).eps}) 
        
    def initialize_placeholders(self):
        # Test data
        self.x_test_tf = tf.placeholder(tf.float32, shape=[None, self.x_test.shape[1]])
        self.y_test_tf = tf.placeholder(tf.float32, shape=[None, self.y_test.shape[1]])
        self.u_test_tf = tf.placeholder(tf.float32, shape=[None, self.u_test.shape[1]])
        
        # Predict data
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x_c.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y_c.shape[1]])

        # Collocation data
        self.x_c_tf = tf.placeholder(tf.float32, shape=[None, self.x_c.shape[1]])
        self.y_c_tf = tf.placeholder(tf.float32, shape=[None, self.y_c.shape[1]])

        # Boundary data
        # left
        self.x_left_tf = tf.placeholder(tf.float32, shape=[None, self.x_left.shape[1]])
        self.y_left_tf = tf.placeholder(tf.float32, shape=[None, self.y_left.shape[1]])
        self.u_left_tf = tf.placeholder(tf.float32, shape=[None, self.u_left.shape[1]])

        # right
        self.x_right_tf = tf.placeholder(tf.float32, shape=[None, self.x_right.shape[1]])
        self.y_right_tf = tf.placeholder(tf.float32, shape=[None, self.y_right.shape[1]])
        self.u_right_tf = tf.placeholder(tf.float32, shape=[None, self.u_right.shape[1]])

        # Lower
        self.x_lower_tf = tf.placeholder(tf.float32, shape=[None, self.x_lower.shape[1]])
        self.y_lower_tf = tf.placeholder(tf.float32, shape=[None, self.y_lower.shape[1]])
        self.u_lower_tf = tf.placeholder(tf.float32, shape=[None, self.u_lower.shape[1]])

        self.x_upper_tf = tf.placeholder(tf.float32, shape=[None, self.x_upper.shape[1]])
        self.y_upper_tf = tf.placeholder(tf.float32, shape=[None, self.y_upper.shape[1]])
        self.u_upper_tf = tf.placeholder(tf.float32, shape=[None, self.u_upper.shape[1]])

    def initialize_session(self):
        tf_config = tf.ConfigProto(allow_soft_placement=True,
                                   log_device_placement=True)
        self.sess = tf.Session(config=tf_config)
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_variables(self):
        # For saving loss
        self.loss_total_log = []
        self.loss_collo_log = []
        self.loss_dir_log = []
        self.loss_neu_log = []
        self.loss_test_log = []
        self.count = 0
        self.newton_started = False

    def net_dnn(self, x, y):
        # Find results
        X = tf.concat([x, y], 1)
        results = self.net_forward(X)

        return results

    def net_forward(self, X):
        num_layers = len(self.weights)+1
        H = X
        # print(H)

        for lyr in range(num_layers-2):
            W = self.weights[lyr]
            b = self.biases[lyr]

            Wx_plus_b = tf.add(tf.matmul(H, W), b)
            H = tf.tanh(Wx_plus_b)

        W = self.weights[-1]
        b = self.biases[-1]
        Y = tf.add(tf.matmul(H, W), b)

        return Y

    def net_physics(self, x, y):
        # Find results from DNN
        T = self.net_dnn(x, y)

        # Temperature gradient
        T_x = tf.gradients(T, x)[0] / self.sigma_x
        T_xx = tf.gradients(T_x, x)[0] / self.sigma_x
        T_y = tf.gradients(T, y)[0] / self.sigma_y
        T_yy = tf.gradients(T_y, y)[0] / self.sigma_y

        x = x*self.sigma_x + self.mu_x
        y = y*self.sigma_y + self.mu_y
        Q = np.zeros(shape=(self.x_c.shape))
        k = self.k
        
        for i in range(0, Q.shape[0]):            
            if 0.2<=self.x_c[i]<=0.3 and 0.2<=self.y_c[i]<=0.3:
                Q[i] = self.Q # W/m^3

        f = k*(T_xx+T_yy) + Q
        
        return f
    
    def normalize_data(self, data, axis):
        if axis == "x":
            normalized_data = (data - self.mu_x) / self.sigma_x
        elif axis == "y":
            normalized_data = (data - self.mu_y) / self.sigma_y

        return normalized_data

    def predict(self, x_star, y_star):
        # Prepare the input
        x_star = (x_star - self.mu_x) / self.sigma_x
        y_star = (y_star - self.mu_y) / self.sigma_y

        # Create dictionary
        tf_dict = {self.x_tf:x_star, self.y_tf:y_star}

        # Predict
        u_star = self.sess.run(self.u_pred, tf_dict)

        return u_star        

    def save_loss(self, file_dir):
        loss_test = np.array(self.loss_test_log)
        loss_data = np.column_stack((self.loss_total_log, 
                                     self.loss_collo_log,
                                     self.loss_dir_log,
                                     self.loss_neu_log,
                                     loss_test))
        loss_df = pd.DataFrame(loss_data, columns=["total", "collo", "dirichlet", "neumann", "error_u"])
        joblib.dump(loss_df, file_dir)

    def save_model(self, file_dir):
        weights = self.sess.run(self.weights)
        biases = self.sess.run(self.biases)
        with open(file_dir, 'wb') as f:
            pickle.dump([weights, biases], f)
            print("Save NN parameters successfully...")

    def unpack(self, data, params):
        # Initialize
        self.data = data
        self.params = params

        # Unpack Parameters
        # Data-Boundary
        self.lb = params["data"]["lb"]
        self.ub = params["data"]["ub"]

        self.random_seed = data["train"]["random_seed"]
        
        # Data-Collocation
        self.x_c = data["train"]["collo"][:, 0:1]
        self.y_c = data["train"]["collo"][:, 1:2]
        self.mu_x = data["train"]["mu_x"]
        self.mu_y = data["train"]["mu_y"]
        self.sigma_x = data["train"]["sigma_x"]
        self.sigma_y = data["train"]["sigma_y"]
        
        # Data-left
        self.x_left = data["train"]["left"][:, 0:1]
        self.y_left = data["train"]["left"][:, 1:2]
        self.u_left = data["train"]["left"][:, 2:3]

        # Data-right
        self.x_right = data["train"]["right"][:, 0:1]
        self.y_right = data["train"]["right"][:, 1:2]
        self.u_right = data["train"]["right"][:, 2:3]
        
        # Data-lower
        self.x_lower = data["train"]["lower"][:, 0:1]
        self.y_lower = data["train"]["lower"][:, 1:2]
        self.u_lower = data["train"]["lower"][:, 2:3]

        # Data-upper
        self.x_upper = data["train"]["upper"][:, 0:1]
        self.y_upper = data["train"]["upper"][:, 1:2]
        self.u_upper = data["train"]["upper"][:, 2:3]

        # Data-Test
        self.x_test = data["test"][:, 0:1]
        self.y_test = data["test"][:, 1:2]
        self.u_test = data["test"][:, 2:3]
        
        # # Physic
        self.k   = params["physic"]["k"]
        self.Q   = params["physic"]["Q"]

        # Network
        self.layers = params["network"]["layers"]
        self.coef_bc_val = np.array(params["network"]["coef_bc"])
        self.verboses_newton = params["network"]["verboses_newton"]
        self.saver = params["network"]["saver"]

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))

        np.random.seed(self.random_seed)
        var = tf.Variable(tf.truncated_normal([in_dim, out_dim], 
                                               stddev=xavier_stddev, 
                                               dtype=tf.float32,
                                               seed=40), 
                           dtype=tf.float32)
        return var


### **Cases**

#### 2D Steady

##### Finite Element (Ref: Zuhal et. al.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import scipy
import warnings
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import time
warnings.filterwarnings("ignore")


class Conduction:
    """
    Solver for 2D heat conduction with heat source on the plate and
    Gaussian random field as the conductivity coefficient.
    """

    def __init__(self, x=(-0.5, 0.5), y=(-0.5, 0.5)):
        """
        Initialize the variables
        Args:
            x (tuple): tuple of x coordinates (x0,x1)
            y (tuple): tuple of x coordinates (y0,y1)
        """
        self.ndimen = 2
        self.x0 = x[0]
        self.y0 = y[0]
        self.x1 = x[1]
        self.y1 = y[1]
        self.length = self.x1 - self.x0
        self.width = self.y1 - self.y0

    def run(self,xi,nx,ny,k,view=True):
        """
        Wrapper to run the code
        Arg:
            - xi (nparray): array of input
        Return:
            - tavgB (float): Average temperature inside region B
        """
        self.k = k
        gridx, gridy = self.creategrid(nx, ny, view=False)
        gx = self.calculatealpha(xi, gridx, gridy, 0.2, view=False)
        self.creatematrix(gx)
        self.solve(view=view)
        tavgB = self.calcB()
        return tavgB

    def creatematrix(self, kx):
        matsize = np.size(self.z,0)
        self.coeffmat = np.zeros(shape=[matsize,matsize])

        # Create lower neumann BC
        for i in range(self.nx+1):
            self.coeffmat[i,i] = 1
            self.coeffmat[i,i+self.nx+1] = -1

        # Create upper dirichlet BC
        for i in range(-(self.nx+1),0):
            self.coeffmat[i,i] = 1

        # Create the remaining
        for i in range(self.nx+1,matsize-(self.nx+1)):
            if i % (self.nx+1) == 0: # left neumann BC
                self.coeffmat[i, i] = 1
                self.coeffmat[i, i + 1] = -1
            elif i % (self.nx+1) == self.nx: # right neumann BC
                self.coeffmat[i, i] = 1
                self.coeffmat[i, i - 1] = -1
            else:
                iloc,jloc = self.z[i,:]
                iloc = int((np.round(iloc,2) + 0.5)*100)
                jloc = int((np.round(jloc,2) + 0.5)*100)
                # k1 = (kx[jloc,iloc+1] - kx[jloc,iloc+1]) / ((2*self.dx)**2)
                # k2 = (kx[jloc+1,iloc] - kx[jloc-1,iloc]) / ((2*self.dy)**2)
                # k3 = kx[jloc,iloc] / (self.dx**2)
                # k4 = kx[jloc, iloc] / (self.dy ** 2)
                k1 = (kx - kx) / ((2*self.dx)**2)
                k2 = (kx - kx) / ((2*self.dy)**2)
                k3 = kx / (self.dx**2)
                k4 = kx / (self.dy ** 2)
                self.coeffmat[i,i] = 2*(k3+k4)
                self.coeffmat[i, i - 1] = -(k3-k1)
                self.coeffmat[i, i + 1] = -(k3+k1)
                self.coeffmat[i, i - (self.nx + 1)] = -(k4-k2)
                self.coeffmat[i, i + (self.nx + 1)] = -(k4+k2)

    def solve(self, view=False):
        self.source = np.zeros(shape=[np.size(self.z,0)])
        for i in range(np.size(self.z,0)):
            x = self.z[i,0]
            y = self.z[i,1]
            if (x >= 0.2 and x <= 0.3) and ((y >= 0.2 and y <= 0.3)):
                self.source[i] = 2000
            else:
                pass

        self.tdist = np.linalg.solve(self.coeffmat,self.source)
        tdist = self.tdist.reshape((self.nx+1, self.ny+1))
        if view:
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3, 3), dpi=300, constrained_layout=False)
            surf = ax.imshow(tdist, cmap=cm.jet, extent=[-0.5, 0.5, -0.5, 0.5], origin='lower', interpolation='bilinear')
            clb = fig.colorbar(surf)
            clb.ax.set_title('T[\u2103]')
            plt.show()

    def calcB(self):
        blist = []
        for i in range(np.size(self.z,0)):
            x = self.z[i, 0]
            y = self.z[i, 1]
            if (x >= -0.3 and x <= -0.2) and ((y >= -0.3 and y <= -0.2)):
                blist.append(self.tdist[i])
            else:
                pass
        blist = np.array(blist)
        TavgB = np.sum(blist*(0.01**2)) / 0.01
        return TavgB


    def calculatealpha(self, xi, gridx, gridy, theta=0.2, view=False):
        grfx, grfy = self.rndfgrid()
        M, li, phii = self.grandomfield(theta,grfx,grfy)
        self.z = np.hstack((gridx.reshape(self.nn, 1), gridy.reshape(self.nn, 1)))
#         gx = np.zeros(shape=[np.size(self.z,0)])
#         for i in range(np.size(self.z,0)):
#             gx[i] = self.calcgz(self.z[i,:],M,xi,li,phii)
#         gx = gx.reshape((np.size(gridx,0), np.size(gridx,1)))

#         if view:
#             fig = plt.figure()
#             ax = fig.gca(projection='3d')
#             surf = ax.plot_surface(gridx, gridy,gx,cmap=cm.coolwarm, linewidth=0, antialiased=False)
#             plt.show()

        # kx = np.exp(1 + gx*0.3)
        kx = self.k
        return kx

    def creategrid(self, nx=100, ny=100, view=False):
        """
        Create discretization for finite difference heat equation solver.
        Args:
             - nx (int): number of spacing on X direction. Default to 100. (number of points is nx+1)
             - ny (int): number of spacing on Y direction. Default to 100. (number of points is ny+1)
             - view (bool): visualize grid or not. Default to False.
         Returns:
             - gridx (nparray): x coordinates for each point across the space.
             - gridy (nparray): y coordinates for each point across the space.
        """
        self.nx = nx
        self.ny = ny
        xx = np.linspace(self.x0, self.x1, nx + 1)
        yy = np.linspace(self.y0, self.y1, ny + 1)
        self.nn = (nx + 1) * (ny + 1)
        gridx, gridy = np.meshgrid(xx, yy)
        self.dx = self.length / nx
        self.dy = self.width / ny

        if view is True:
            for i in range(nx + 1):
                temp1 = np.array([[-0.5, yy[i]], [0.5, yy[i]]])
                temp2 = np.array([[xx[i], -0.5], [xx[i], 0.5]])
                plt.plot(temp1[:, 0], temp1[:, 1], 'k')
                plt.plot(temp2[:, 0], temp2[:, 1], 'k')
            # plt.scatter(gridx, gridy)
            plt.show()

        return gridx, gridy

    def rndfgrid(self, nx=10, ny=10, view=False):
        """
        Create RGF nodes in the domain.
        Args:
            nx (int): number of spacing on X direction. Default to 10. (number of points is nx+1)
            ny (int): number of spacing on Y direction. Default to 10. (number of points is ny+1)
            view (bool): display nodes or not.
        Returns:
             - rf_gridx (nparray): x coordinates for each point across the space.
             - rf_gridy (nparray): y coordinates for each point across the space.
        """
        rf_xx = np.linspace(self.x0, self.x1, nx+1)
        rf_yy = np.linspace(self.y0, self.y1, ny+1)
        self.rf_nn = (nx+1) * (ny+1)
        rf_gridx, rf_gridy = np.meshgrid(rf_xx, rf_yy)

        if view is True:
            for i in range(nx+1):
                temp1 = np.array([[-0.5, rf_yy[i]], [self.rf_yy[i]]])
                temp2 = np.array([[rf_xx[i], -0.5], [rf_xx[i], 0.5]])
                plt.plot(temp1[:, 0], temp1[:, 1], 'k')
                plt.plot(temp2[:, 0], temp2[:, 1], 'k')
            plt.scatter(rf_gridx, rf_gridy)
            plt.show()

        return rf_gridx,rf_gridy

    def grandomfield(self, theta, rf_gridx, rf_gridy):
        """
        Calculate the components inside the Gaussian random field.
        Args:
            - theta (float): Lengthscale of Kernel function
            - rf_gridx (nparray):  x coordinates for each point of GRF grid across the space.
            - rf_gridy (nparray):  y coordinates for each point of GRF grid across the space.
        Returns:
            - M (int): Number of dimension of the input variables.
            - li (nparray): Eigenvalues of correlation matrix.
            - phii (nparray): Eigenvectors of correlation matrix.
        """
        self.theta = theta * np.ones(shape=[self.ndimen])
        self.zeta = np.hstack((rf_gridx.reshape(self.rf_nn, 1), rf_gridy.reshape(self.rf_nn, 1,)))
        c_zetazeta = kernel(self.zeta, self.zeta, self.ndimen, self.theta)
        li, phii = np.linalg.eigh(c_zetazeta)
        li = np.flip(li,0)
        phii = np.flip(phii,1)

        for M in range(1,self.rf_nn+1):
            temp1 = np.sum(li[:M]) / np.sum(li)
            if temp1 >= 0.99:
                break
            else:
                pass

        return M,li,phii

#     def calcgz(self,z,M,xi,li,phii):
#         c_zzeta = kernel(z, self.zeta, self.ndimen, self.theta).transpose()
#         gztemp = np.zeros(shape=[M])
#         for i in range(M):
#             gztemp[i] = (xi[i]/np.sqrt(li[i])) * np.dot(phii[:,i], c_zzeta)
#         gz = np.sum(gztemp)
#         return gz

    def basisfunc(self,z,i,phii):
        c_zzeta = kernel(z, self.zeta, self.ndimen, self.theta).transpose()
        gztemp = np.dot(phii[:,i], c_zzeta)
        return gztemp

def kernel(XN, XM, nvar, theta):
    if XN.ndim == 1:
        XN = np.array([XN])
    mdist = np.zeros((np.size(XN, 0), np.size(XM, 0), nvar))
    for ii in range(0, nvar):
        X1 = np.transpose(np.array([XN[:, ii]]))
        X2 = np.transpose(np.array([XM[:, ii]]))
        mdist[:, :, ii] = (cdist(X1, X2, 'euclidean') ** 2) / (theta[ii] ** 2)
    Psi = np.exp(-1 * np.sum(mdist, 2))
    return Psi

# if __name__ == "__main__":
#     case = 2
#     if case == 1:
#         test = Conduction()
#         gridx, gridy = test.creategrid(100, 100, view=False)
#         grfx, grfy = test.rndfgrid()
#         z = np.hstack((gridx.reshape(test.nn, 1), gridy.reshape(test.nn, 1)))
#         M, li, phii = test.grandomfield(0.2, grfx, grfy)
#         gx = np.zeros(shape=[np.size(z, 0)])
#         fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(11, 11), subplot_kw={'xticks': [], 'yticks': []})
#         for j in range(20):
#             ax = axs.flat[j]
#             for i in range(np.size(z, 0)):
#                 gx[i] = test.basisfunc(z[i, :], j, phii)
#             gxi = gx.reshape((np.size(gridx,0), np.size(gridx,1)))
#             surf = ax.imshow(gxi, cmap=cm.jet, extent=[-0.5, 0.5, -0.5, 0.5], origin='lower', interpolation='bilinear')
#         plt.tight_layout()
#         plt.show()
#     else:
#         xi = 1*np.random.randn(53)
#         t = time.time()
#         plate = Conduction()
#         tavgB = plate.run(xi, view=True)
#         print(tavgB)
#         elapsed = time.time() - t
#         print(elapsed,'s')



##### PINN

In [ ]:
class Plate:
    def __init__(self, add_noise=False, save_fig=False):
        self.add_noise = add_noise
        self.save_fig = save_fig
        self.data = {}

    def generate_bc(self, loc):
        if loc == "left":
            n = self.n_left
            x = np.ones((n,1)) * self.lb[0]
            np.random.seed(self.random_seed)
            y = lhs(1,n) * (self.ub[1] - self.lb[1]) + self.lb[1]
            q = self.q_left*lhs(1,n)

            bc_datas = np.concatenate((x, y, q), axis=1)
        elif loc == "right":
            n = self.n_right
            x = np.ones((n,1)) * self.ub[0]
            np.random.seed(self.random_seed)
            y = lhs(1,n) * (self.ub[1] - self.lb[1]) + self.lb[1]
            q = self.q_right*lhs(1,n)
            
            bc_datas = np.concatenate((x, y, q), axis=1)
        elif loc == "lower":
            n = self.n_lower
            np.random.seed(self.random_seed)
            x = lhs(1,n) * (self.ub[0] - self.lb[0]) + self.lb[0]
            y = np.ones((n,1)) * self.lb[1]
            q = self.q_lower*lhs(1,n)

            bc_datas = np.concatenate((x, y, q), axis=1)
        else:
            n = self.n_upper
            np.random.seed(self.random_seed)
            x = lhs(1,n) * (self.ub[0] - self.lb[0]) + self.lb[0]
            y = np.ones((n,1)) * (self.ub[1])
            np.random.seed(self.random_seed)
            T = self.T_upper*lhs(1,n)

            bc_datas = np.concatenate((x, y, T), axis=1)
        
        return bc_datas

    def generate_collo(self):
        # unpack
        n = self.n_collo
        lb = self.lb
        ub = self.ub

        outer = 0.05*2
        lb_sc = self.lb_source - outer
        ub_sc = self.ub_source + outer

        # # create points
        np.random.seed(self.random_seed)
        collo_pts = lb + (ub-lb)*lhs(2, n, criterion=None)
        
        if self.model_type == "optimized":
            collo_pts_ = []
            for i in range(0, n):
                if (collo_pts[i, 0] <= lb_sc[0] or collo_pts[i, 0] >= ub_sc[0]) or (collo_pts[i, 1] <= lb_sc[1] or collo_pts[i, 1] >= ub_sc[1]):
                    collo_pts_.append(collo_pts[i,:])

            np.random.seed(self.random_seed)
            inner_collo = lb_sc + (ub_sc - lb_sc)*lhs(2, self.n_inner_collo)
            collo_pts_new = np.concatenate((np.array(collo_pts_), inner_collo), axis=0)
        elif self.model_type == "baseline": 
            collo_pts_new=collo_pts
            
        self.mu_X, self.sigma_X = collo_pts_new.mean(0), collo_pts_new.std(0)
        self.mu_x, self.sigma_x = self.mu_X[0], self.sigma_X[0]
        self.mu_y, self.sigma_y = self.mu_X[1], self.sigma_X[1]
                
        return (collo_pts_new) 

    def generate_train_data(self, param):
        
        # Unpack parameters
        self.unpack(param)

        # Generate points
        # Generate boundary conditions data 
        left_bc = self.generate_bc('left')
        right_bc = self.generate_bc('right')
        lower = self.generate_bc('lower')
        upper = self.generate_bc('upper')

        # Generate collocations points data
        collo_ = self.generate_collo()

        # Pack data
        self.data["train"] = {}
        self.data["train"]["collo"] = collo_
        self.data["train"]["left"] = left_bc
        self.data["train"]["right"] = right_bc
        self.data["train"]["lower"] = lower
        self.data["train"]["upper"] = upper

        self.data["train"]["mu_x"] = self.mu_x
        self.data["train"]["sigma_x"] = self.sigma_x
        self.data["train"]["mu_y"] = self.mu_y
        self.data["train"]["sigma_y"] = self.sigma_y

        self.data["train"]["random_seed"] = self.random_seed
    
    def generate_test_data(self, param):
        plate = Conduction()
        np.random.seed(self.random_seed)
        xi = 1*np.random.randn(53)
        k = self.k_
        tavgB = plate.run(xi=xi, nx=100, ny=100, k=k, view=True)
        X_flat = plate.z[:, 0]
        Y_flat = plate.z[:, 1]
        T_ = plate.tdist

        self.data["test"] = np.column_stack((X_flat, Y_flat, T_))

    def plot(self):
        # unpack data
        collo_ = self.data["train"]["collo"]
        left_ = self.data["train"]["left"]
        right_ = self.data["train"]["right"]
        lower_ = self.data["train"]["lower"]
        upper_ = self.data["train"]["upper"]

        # PLOT: points distribution
        # Properties
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')

        # Plot
        fig_w = 10
        fig_h = 10
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), constrained_layout=True, dpi=300)

        ax.plot([self.lb[0], self.ub[0]], [self.lb[1], self.lb[1]], 'k')
        ax.plot([self.lb[0], self.ub[0]], [self.ub[1], self.ub[1]], 'k')
        ax.plot([self.lb[0], self.lb[0]], [self.ub[1], self.lb[1]], 'k')
        ax.plot([self.ub[0], self.ub[0]], [self.ub[1], self.lb[1]], 'k')

        ax.plot([0.2, 0.3], [0.2, 0.2], 'm')
        ax.plot([0.3, 0.3], [0.2, 0.3], 'm')
        ax.plot([0.3, 0.2], [0.3, 0.3], 'm')
        ax.plot([0.2, 0.2], [0.3, 0.2], 'm')

        ax.scatter(collo_[:,0:1], collo_[:,1:2], marker='.', alpha=0.7, c='grey', label='collo')
        ax.scatter(left_[:,0:1], left_[:,1:2], marker='.', alpha=1.0, c='r', label='left BC')
        ax.scatter(right_[:,0:1], right_[:,1:2], marker='.', alpha=1.0, c='g', label='right BC')
        ax.scatter(lower_[:,0:1], lower_[:,1:2], marker='.', alpha=1.0, c='b', label='lower BC')
        ax.scatter(upper_[:,0:1], upper_[:,1:2], marker='.', alpha=1.0, c='yellow', label='Upper BC')
        
        ax.set_title("Points distribution", fontsize=40)
        #ax.set_xlabel("$x$ (m)", fontsize=20)
        #ax.set_ylabel("$y$ (m)", fontsize=20)
        ax.set_xlabel("$x$ (m)", fontsize=25)
        ax.set_ylabel("$y$ (m)", fontsize=25)
        ax.tick_params(axis='both', which='major', labelsize=20)
        ax.legend(fontsize=20, loc=4)
        ax.grid(linestyle="--")
        ax.set_xlim(self.lb[0], self.ub[0])
        ax.set_ylim(self.lb[1], self.ub[1])

        if self.save_fig:
            fig.savefig("fig_point_distribution.eps", format="eps")
        plt.show()

        if self.save_fig:
            fig.savefig("fig_references.eps", format="eps")
        plt.show()

    def unpack(self, param):

        # Bound
        self.lb = param["data"]["lb"]
        self.ub = param["data"]["ub"]
        self.lb_source = param["data"]["lb_source"]
        self.ub_source = param["data"]["ub_source"]
        
        self.model_type = param["data"]["model_type"]

        # Discretizations
        self.n_collo = param["data"]["n_collo"]
        if self.model_type == "optimized":
            self.n_inner_collo = param["data"]["inner_collo"]
        self.n_left = param["data"]["n_left"]
        self.n_right = param["data"]["n_right"]
        self.n_lower = param["data"]["n_lower"]
        self.n_upper = param["data"]["n_upper"]
        self.n_test = param["data"]["n_test"]

        self.q_left = param["data"]["left_q"]
        self.q_right = param["data"]["right_q"]
        self.q_lower = param["data"]["lower_q"]
        self.T_upper = param["data"]["upper_T"]
        
        self.random_seed = param["data"]["seed"]
        
        self.k_ = param["physic"]["k"]

### **Post Processing**

In [ ]:
class PostProcessing:

    def __init__(self, model, params, save_fig=False):
        self.save_fig = save_fig
        self.unpack(model, params)

    def calculate_pinn(self):
        
        self.n_x = self.n_test
        self.n_y = self.n_test
        self.u_pinn = self.model.predict(self.X_flat, self.Y_flat)
        self.x_pinn_ = self.X_flat.reshape(self.n_y, self.n_x)
        self.y_pinn_ = self.Y_flat.reshape(self.n_y, self.n_x)
        self.u_pinn_ = self.u_pinn.reshape(self.n_y, self.n_x)

    def calculate_error_plate(self):
        x_test = self.model.x_test
        y_test = self.model.y_test
        u_test = self.model.u_test
        
        u_pred = self.model.predict(x_test, y_test)

        delta_u = np.abs(u_pred - u_test)
        self.abs_err_u = np.sum(delta_u) / (x_test.shape[0])

        rel_u = delta_u / (u_test)
        self.rel_err_u = np.sum(rel_u) / (x_test.shape[0])

        delta_squared = delta_u**2
        self.rmse_u = np.sqrt(np.sum(delta_squared) / x_test.shape[0])
                
        self.mean_u_test = np.mean(u_test)
        self.SST = np.sum((u_test - self.mean_u_test)**2)
        self.SSE = np.sum(delta_squared)
        self.R2 = 1 - self.SSE / self.SST
        
        print(f"Absolute Error: {self.abs_err_u:5f}")
        print(f"Relative Error (%): {self.rel_err_u*100:5f}")
        print(f"RMSE: {self.rmse_u:5f}")
        print(f"- R2: {self.R2:5f}")

    def create_test_data(self):
        # Find fraction

        self.x = np.linspace(self.lb[0], self.ub[0], num=self.n_test)
        self.y = np.linspace(self.lb[1], self.ub[1], num=self.n_test)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        self.X_flat = self.model.x_test
        self.Y_flat = self.model.y_test

    def display_loss(self):
        # SET
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')

        # Extract data
        loss_total = np.sqrt(self.model.loss_total_log)
        loss_collo = np.sqrt(self.model.loss_collo_log)
        loss_test = self.model.loss_test_log
        loss_dir = np.sqrt(self.model.loss_dir_log)
        loss_neu = np.sqrt(self.model.loss_neu_log)

        if self.model.newton_started:
            run_status = "newton"
        else:
            run_status = "none"

        # Prepare
        iter_total = []

        if run_status == "newton":
            n_total = np.array(loss_total).shape[0]

            for i in range(n_total):
                iter_total.append(i)
        else:
            n_total = 0
        
        # PLOT History
        if run_status != "none":
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6), constrained_layout=True, dpi=300)
            ax.plot(iter_total, loss_total, "r", linestyle="solid", label="Physical Loss")
            ax.plot(iter_total, loss_collo, "g", linestyle="dashdot", label="Collocation Loss")
            ax.plot(iter_total, loss_test, "black", linestyle="dashed", label="Test Error")
            ax.plot(iter_total, loss_dir, "b", linestyle="dotted", label="Dirichlet Boundary Loss")
            ax.plot(iter_total, loss_neu, 'purple',linestyle='solid', label='Neumann Boundary Loss')

            ax.set_xlim(0, iter_total[-1])
            ax.set_yscale("log")
            ax.set_xlabel("Iterations", fontsize=25)
            ax.set_ylabel("RMSE", fontsize=25)
            ax.grid(linestyle="--")
            ax.legend(fontsize=13)
            ax.set_title('Loss History (Log Scale)', fontsize=35)
            ax.tick_params(axis='both', which='major', labelsize=11)

            # save figure
            if self.save_fig:
                fig.savefig("fig_loss_history.eps", format="eps")
            plt.show()
        
        print(f"- Last Iterations: {iter_total[-1]}")
        
    def display_contour(self):
        # FIND: PINN results
        self.create_test_data()
        self.calculate_pinn()

        # PLOT CONTOUR
        fig_h = 5
        fig_w = 5

        # CALCULATE: error
        self.calculate_error_plate()
        
        fig_w = fig_h

        v_min = np.round(np.min(self.u_pinn_))
        v_max = np.round(np.max(self.u_pinn_))
        self.plot_countour(phi_=self.u_pinn_,fig_w=fig_w, fig_h=fig_h, vmin=v_min, vmax=v_max, title="PINN Solution")
        
        
        u_test = (self.model.u_test)
        u_test = u_test.reshape(self.n_x,self.n_y)
        phi = np.abs(self.u_pinn_ - u_test)
        v_min = 0.
        v_max = (np.max(phi))
        self.plot_countour2(phi_= phi,fig_w=fig_w, fig_h=fig_h, vmin=v_min, vmax=v_max, title="Absolute Error")

        

    def plot_countour(self,phi_,fig_w, fig_h, vmin, vmax, title):
        # Set
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')
        
        # Create plot
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), dpi=300, constrained_layout=False)

        # Unpack
        cf = ax.imshow(phi_, cmap=cm.jet, extent=[-0.5, 0.5, -0.5, 0.5], origin='lower', interpolation='bilinear')
        ax.set_xlim(self.lb[0], self.ub[0])
        ax.set_ylim(self.lb[1], self.ub[1])
        
        ax.set_title(title, fontsize=25)
        
        ax.set_xlabel('$x$ (m)', fontsize=20)
        ax.set_ylabel('$y$ (m)', fontsize=20)
        ax.tick_params(axis='both', which='major', labelsize=15)
        # ax.contour(x, y, phi_, colors='k', linewidths=0.2, levels=50)
        
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="2%", pad=0.1)
        cb = fig.colorbar(cf, cax=cax)
        
        ticks = np.linspace(vmin, vmax, 3)
        ticks = np.round(ticks, 2)
        cb.set_ticks(ticks)
        cb.ax.tick_params(labelsize=16)
        cb.ax.set_title("$T$ [\u2103]", fontsize=20)

        plt.show()

        # save figure
        if self.save_fig:
            fig.savefig(f"fig_{types}.eps", format="eps")
        plt.show()
        
    def plot_countour2(self,phi_,fig_w, fig_h, vmin, vmax, title):
        # Set
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')
        
        # Create plot
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(fig_w, fig_h), dpi=300, constrained_layout=False)

        # Unpack
        cf = ax.imshow(phi_, cmap=cm.jet, extent=[-0.5, 0.5, -0.5, 0.5], origin='lower', interpolation='bilinear')
        ax.set_xlim(self.lb[0], self.ub[0])
        ax.set_ylim(self.lb[1], self.ub[1])
        
        ax.set_title(title, fontsize=25)
        
        ax.set_xlabel('$x$ (m)', fontsize=20)
        ax.set_ylabel('$y$ (m)', fontsize=20)
        ax.tick_params(axis='both', which='major', labelsize=15)
        # ax.contour(x, y, phi_, colors='k', linewidths=0.2, levels=50)
        
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="2%", pad=0.1)
        cb = fig.colorbar(cf, cax=cax)
        
        ticks = np.linspace(vmin, vmax, 3)
        ticks = np.round(ticks, 2)
        ticks[-1] = ticks[-1] - 0.01
        ticks[0] = ticks[0] + 0.01
        cb.set_ticks(ticks)
        cb.ax.tick_params(labelsize=16)
        cb.ax.set_title("$T$ [\u2103]", fontsize=20)

        plt.show()

    def unpack(self, model, params):
        self.model = model
        self.params = params

        self.lb = params["data"]["lb"]
        self.ub = params["data"]["ub"]
        self.n_test = params["data"]["n_test"]
        self.coef_bc = params["network"]["coef_bc"]
        self.verboses_newton = params["network"]["verboses_newton"]


# **RUN**

In [ ]:

with tf.device('/device:GPU:1'):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)



## **Heat Conduction**

In [ ]:
def generate_param():
  # ganti learning rate

    params = {}
    params["data"] = {}
    n = 176
    params["data"]["n_left"] = n #81
    params["data"]["n_right"] = n
    params["data"]["n_lower"] = n
    params["data"]["n_upper"] = n
    params["data"]["n_test"] = 101
    params["data"]["lb"] = np.array([-0.5, -0.5])
    params["data"]["ub"] = np.array([0.5, 0.5])
    params["data"]["lb_source"] = np.array([0.2, 0.2])
    params["data"]["ub_source"] = np.array([0.3, 0.3])
    
    params["data"]["left_q"] = 0 
    params["data"]["right_q"] = 0 
    params["data"]["lower_q"] = 0
    params["data"]["upper_T"] = 0
    seed = np.random.randint(1,10000)
    params["data"]["seed"] = seed
    print(f"Seed: {seed}")

    params["physic"] = {}
    params["physic"]["Q"]   = 2000

    params["network"] = {}
    params["network"]["coef_bc"] = 0.001
    params["network"]["verboses_adam"] = 100
    params["network"]["verboses_newton"] = 100
    params["network"]["saver"] = 5000
    params["network"]["batches"] = 1000
    params["network"]["lr_init"] = 1e-3
    params["network"]["epochs"] = 1001 #10001
    

    return params


#### **2D Steady Plate**

##### **Generate Parameters**

In [ ]:
# choose model
model_type = "optimized"
# model_type = "baseline"

params = generate_param()
params["data"]["model_type"] = model_type
params["network"]["layers"] = [2] + 5*[30] + [1]
params["physic"]["k"] = 3

if model_type == "optimized":
    params["network"]["coef_bc"] = 0.001
    params["data"]["n_collo"] = 7000
    params["data"]["inner_collo"] = 1000
elif model_type == "baseline":
    params["network"]["coef_bc"] = 1.0
    params["data"]["n_collo"] = 10000    

case = Plate()
case.generate_train_data(param=params)
case.plot()


##### Generate Test Data

In [ ]:
case.generate_test_data(param=params)

##### **Run**

###### Initial

In [ ]:
# Create Model Instance
model = Pinn(data=case.data, params=params)

In [ ]:
  # # Fit using BFGS-B
model.fit_newton()

In [ ]:
# Create Results
results = PostProcessing(model=model,
                      params=params,
                      save_fig=False)

results.display_loss()
results.display_contour()